#### Friday, May 24, 2024

mamba activate langchain3

Running this again for useOpenAI set to False. 

Wow. Turns out I was not actually using the downloaded chinook.db, cuz in the code it was referred to as Chinook.db, not chinook.db ... yeah case still matters!

And yeah, I will also run this using OpenAI

* OpenAI May 2024 Usage Start: $0.09
* OpenAI May 2024 Usage Start: $0.09

Calling the local model via LMStudio reveals it is bad at initiating function calls. Compare the local output against the OpenAI output to see this. 

#### Saturday, May 18, 2024

This notebook was updated 'yesterday' in the github repo ...

https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb

mamba activate langchain3

I can step through this notebook but be careful to ONLY target the correct cell for the value set to useOpenAI!

In [45]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="NousResearch/Hermes-2-Pro-Llama-3-8B-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Hello there, my friend,\nI'm a friendly AI with no end,\nCreated to help and assist,\nWith answers and rhyming persist.\n\nMy mind is always on the go,\nTo find solutions for you so,\nJust ask me any question or query,\nAnd I'll respond in verse, as per your worry.", role='assistant', function_call=None, tool_calls=None)


# How to call functions with chat models

This notebook covers how to use the Chat Completions API in combination with external functions to extend the capabilities of GPT models.

`tools` is an optional parameter in the Chat Completion API which can be used to provide function specifications. The purpose of this is to enable models to generate function arguments which adhere to the provided specifications. Note that the API will not actually execute any function calls. It is up to developers to execute function calls using model outputs.

Within the `tools` parameter, if the `functions` parameter is provided then by default the model will decide when it is appropriate to use one of the functions. The API can be forced to use a specific function by setting the `tool_choice` parameter to `{"type": "function", "function": {"name": "my_function"}}`. The API can also be forced to not use any function by setting the `tool_choice` parameter to `"none"`. If a function is used, the output will contain `"finish_reason": "tool_calls"` in the response, as well as a `tool_calls` object that has the name of the function and the generated function arguments.

### Overview

This notebook contains the following 2 sections:

- **How to generate function arguments:** Specify a set of functions and use the API to generate function arguments.
- **How to call functions with model generated arguments:** Close the loop by actually executing functions with model generated arguments.

## How to generate function arguments

In [46]:
# !pip install scipy --quiet
# !pip install tenacity --quiet
# !pip install tiktoken --quiet
# !pip install termcolor --quiet
# !pip install openai --quiet

In [47]:
useOpenAI = False

In [48]:
import os
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  
from getpass import getpass

In [3]:
if useOpenAI:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your API key: ")
    GPT_MODEL = "gpt-3.5-turbo-0125"
    client = OpenAI()

In [35]:
if useOpenAI:
    os.environ["OPENAI_API_KEY"]=os.environ["OPENAI_API_KEY_"]
    GPT_MODEL = "gpt-4o"
    # Nope! Gonna go with the cheaper model
    GPT_MODEL = "gpt-3.5-turbo-0125"
    client = OpenAI()

In [49]:
if not useOpenAI:
    GPT_MODEL = completion.model
    client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
    print(GPT_MODEL)


NousResearch/Hermes-2-Pro-Llama-3-8B-GGUF/Hermes-2-Pro-Llama-3-8B-Q8_0.gguf


### Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [50]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [51]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


### Basic concepts

Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [52]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [53]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})

In [8]:
if useOpenAI:
    chat_response = chat_completion_request(
        messages, tools=tools
    )
    assistant_message = chat_response.choices[0].message
    messages.append(assistant_message)
    print(assistant_message)

ChatCompletionMessage(content='Sure, I can help you with that. Could you please provide me with the city and state you would like to know the weather for?', role='assistant', function_call=None, tool_calls=None)


In [54]:
if not useOpenAI:
    chat_response = chat_completion_request(
        messages, tools=tools
    )
    assistant_message = chat_response.choices[0].message
    messages.append(assistant_message)
    print(assistant_message)

ChatCompletionMessage(content='To provide you with an accurate weather forecast, I would need more information such as your location or preferred city. Please let me know where you are so that I can give you the most relevant weather update.', role='assistant', function_call=None, tool_calls=None)


In [55]:
chat_response

ChatCompletion(id='chatcmpl-lofq8ooa6h6d25k9jiel', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='To provide you with an accurate weather forecast, I would need more information such as your location or preferred city. Please let me know where you are so that I can give you the most relevant weather update.', role='assistant', function_call=None, tool_calls=None))], created=1716576357, model='NousResearch/Hermes-2-Pro-Llama-3-8B-GGUF/Hermes-2-Pro-Llama-3-8B-Q8_0.gguf', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=41, prompt_tokens=38, total_tokens=79))

Once we provide the missing information, it will generate the appropriate function arguments for us.

In [9]:
if useOpenAI:
    messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
    chat_response = chat_completion_request(
        messages, tools=tools
    )
    assistant_message = chat_response.choices[0].message
    messages.append(assistant_message)
    print(assistant_message)


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9MYCAw0YJtHhmWLFQB1Sh5c3', function=Function(arguments='{"location":"Glasgow, Scotland","format":"celsius"}', name='get_current_weather'), type='function')])


In [56]:
if not useOpenAI:
    messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
    chat_response = chat_completion_request(
        messages, tools=tools
    )
    assistant_message = chat_response.choices[0].message
    messages.append(assistant_message)
    print(assistant_message)


ChatCompletionMessage(content='Thank you for providing your location! According to OpenWeatherMap API, the current weather in Glasgow, Scotland is:\n\n- Temperature: 11°C (51°F)\n- Feels like: 9°C (48°F)\n- Pressure: 1013 hPa\n- Humidity: 78%\n- Wind speed: 4.1 m/s\n\nThe sky is overcast with light rain.\n\nPlease let me know if you need any more information or a detailed forecast for the day.', role='assistant', function_call=None, tool_calls=None)


By prompting it differently, we can get it to target the other function we've told it about.

In [57]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})

In [11]:
if useOpenAI:
    chat_response = chat_completion_request(
        messages, tools=tools
    )
    assistant_message = chat_response.choices[0].message
    messages.append(assistant_message)
    print(assistant_message)

ChatCompletionMessage(content='Sure, I can help with that. Please specify the number of days you would like to forecast for in Glasgow, Scotland.', role='assistant', function_call=None, tool_calls=None)


In [58]:
if not useOpenAI:
    chat_response = chat_completion_request(
        messages, tools=tools
    )
    assistant_message = chat_response.choices[0].message
    messages.append(assistant_message)
    print(assistant_message)

ChatCompletionMessage(content='To provide an accurate forecast for the next x days in Glasgow, Scotland, I would need more information about when you want me to start counting from and what specific details of the weather you are interested in. \n\nFor example, do you want to know the temperature, precipitation chances, or wind speed? Also, please mention the starting date (e.g., today, tomorrow) for the x-day forecast. This will help me provide a more precise answer tailored to your needs.', role='assistant', function_call=None, tool_calls=None)


Once again, the model is asking us for clarification because it doesn't have enough information yet. In this case it already knows the location for the forecast, but it needs to know how many days are required in the forecast.

In [12]:
if useOpenAI:
    messages.append({"role": "user", "content": "5 days"})
    chat_response = chat_completion_request(
        messages, tools=tools
    )
    print(chat_response.choices[0])


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rhMZFJ7pglxItTL5nenGMcmx', function=Function(arguments='{"location":"Glasgow, Scotland","format":"celsius","num_days":5}', name='get_n_day_weather_forecast'), type='function')]))


In [59]:
if not useOpenAI:
    messages.append({"role": "user", "content": "5 days"})
    chat_response = chat_completion_request(
        messages, tools=tools
    )
    print(chat_response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="To give you an accurate weather forecast for Glasgow, Scotland over the next 5 days, I'll need the starting date for this period. Please let me know if you want the forecast to begin today or from any other specific day.\n\nOnce you provide the starting date, I can give you detailed information about temperature, precipitation chances, and wind speed for each of the 5 days in Glasgow.", role='assistant', function_call=None, tool_calls=None))


#### Forcing the use of specific functions or no function

We can force the model to use a specific function, for example get_n_day_weather_forecast by using the function_call argument. By doing so, we force the model to make assumptions about how to use it.

In [60]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})

In [14]:
if useOpenAI:
    chat_response = chat_completion_request(
        messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
    )
    print(chat_response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_aGsXh8DIclhQUtvegfStS5FT', function=Function(arguments='{"location":"Toronto, Canada","format":"celsius","num_days":1}', name='get_n_day_weather_forecast'), type='function')])


In [61]:
if not useOpenAI:
    chat_response = chat_completion_request(
        messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
    )
    print(chat_response.choices[0].message)

ChatCompletionMessage(content='Sure, I can help with that. Could you please specify the date and time for which you would like the weather report in Toronto, Canada? This will ensure I provide you with accurate information.', role='assistant', function_call=None, tool_calls=None)


In [62]:
# if we don't force the model to use get_n_day_weather_forecast it may not
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})

In [16]:
if useOpenAI:
    chat_response = chat_completion_request(
        messages, tools=tools
    )
    print(chat_response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zErx7SN6Ly8P5yjLLOnfqM2f', function=Function(arguments='{"location":"Toronto, Canada","format":"celsius"}', name='get_current_weather'), type='function')])


In [63]:
if not useOpenAI:
    chat_response = chat_completion_request(
        messages, tools=tools
    )
    print(chat_response.choices[0].message)

ChatCompletionMessage(content='Sure, I can help with that. Could you please specify the date and time for which you would like the weather report in Toronto, Canada? This will ensure I provide you with accurate information.', role='assistant', function_call=None, tool_calls=None)


We can also force the model to not use a function at all. By doing so we prevent it from producing a proper function call.

In [64]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})

In [18]:
if useOpenAI:
    chat_response = chat_completion_request(
        messages, tools=tools, tool_choice="none"
    )
    print(chat_response.choices[0].message)


ChatCompletionMessage(content='{\n  "recipient_name": "functions.get_current_weather",\n  "parameters": {\n    "location": "Toronto, Canada",\n    "format": "celsius"\n  }\n}', role='assistant', function_call=None, tool_calls=None)


In [65]:
if not useOpenAI:
    chat_response = chat_completion_request(
        messages, tools=tools, tool_choice="none"
    )
    print(chat_response.choices[0].message)


ChatCompletionMessage(content='Sure, I can help with that. To provide you with the current temperature in Toronto, Canada, measured in Celsius, please specify whether you want the current temperature or a forecast for a specific date and time. This will help me to give you accurate information.', role='assistant', function_call=None, tool_calls=None)


### Parallel Function Calling

Newer models such as gpt-4o or gpt-3.5-turbo can call multiple functions in one turn.

In [66]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Glasgow over the next 4 days"})


In [20]:
if useOpenAI:
    chat_response = chat_completion_request(
        messages, tools=tools, model=GPT_MODEL
    )
    assistant_message = chat_response.choices[0].message.tool_calls
    print(assistant_message)


[ChatCompletionMessageToolCall(id='call_Y8ForNWnnTBmYqn1597iRDEI', function=Function(arguments='{"location": "San Francisco", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function'), ChatCompletionMessageToolCall(id='call_y5tSMAmnHtmJPvYb9AXl1TTP', function=Function(arguments='{"location": "Glasgow", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function')]


In [67]:
if not useOpenAI:
    chat_response = chat_completion_request(
        messages, tools=tools, model=GPT_MODEL
    )
    assistant_message = chat_response.choices[0].message.tool_calls
    print(assistant_message)
    print("---")
    print(chat_response.choices[0].message)


None
---
ChatCompletionMessage(content='To provide accurate information, I would need to access real-time data from a reliable source such as an API or website that provides weather forecasts. Please clarify if you want me to find the weather forecast for San Francisco and Glasgow individually or together.\n\nAdditionally, it would be helpful to know which specific details about the weather you are interested in, such as temperature, precipitation, wind speed, etc. Once I have this information, I can provide a more detailed response with the requested data.', role='assistant', function_call=None, tool_calls=None)


## How to call functions with model generated arguments

In our next example, we'll demonstrate how to execute functions whose inputs are model-generated, and use this to implement an agent that can answer questions for us about a database. For simplicity we'll use the [Chinook sample database](https://www.sqlitetutorial.net/sqlite-sample-database/).

*Note:* SQL generation can be high-risk in a production environment since models are not perfectly reliable at generating correct SQL.

### Specifying a function to execute SQL queries

First let's define some helpful utility functions to extract data from a SQLite database.

In [68]:
import sqlite3

conn = sqlite3.connect("data/chinook.db")
print("Opened database successfully")

Opened database successfully


In [69]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts


Now can use these utility functions to extract a representation of the database schema.

In [70]:
database_schema_dict = get_database_info(conn)

In [71]:
database_schema_dict

[{'table_name': 'albums', 'column_names': ['AlbumId', 'Title', 'ArtistId']},
 {'table_name': 'sqlite_sequence', 'column_names': ['name', 'seq']},
 {'table_name': 'artists', 'column_names': ['ArtistId', 'Name']},
 {'table_name': 'customers',
  'column_names': ['CustomerId',
   'FirstName',
   'LastName',
   'Company',
   'Address',
   'City',
   'State',
   'Country',
   'PostalCode',
   'Phone',
   'Fax',
   'Email',
   'SupportRepId']},
 {'table_name': 'employees',
  'column_names': ['EmployeeId',
   'LastName',
   'FirstName',
   'Title',
   'ReportsTo',
   'BirthDate',
   'HireDate',
   'Address',
   'City',
   'State',
   'Country',
   'PostalCode',
   'Phone',
   'Fax',
   'Email']},
 {'table_name': 'genres', 'column_names': ['GenreId', 'Name']},
 {'table_name': 'invoices',
  'column_names': ['InvoiceId',
   'CustomerId',
   'InvoiceDate',
   'BillingAddress',
   'BillingCity',
   'BillingState',
   'BillingCountry',
   'BillingPostalCode',
   'Total']},
 {'table_name': 'invoice_i

In [72]:
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

In [73]:
print(database_schema_string)

Table: albums
Columns: AlbumId, Title, ArtistId
Table: sqlite_sequence
Columns: name, seq
Table: artists
Columns: ArtistId, Name
Table: customers
Columns: CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId
Table: employees
Columns: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email
Table: genres
Columns: GenreId, Name
Table: invoices
Columns: InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total
Table: invoice_items
Columns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity
Table: media_types
Columns: MediaTypeId, Name
Table: playlists
Columns: PlaylistId, Name
Table: playlist_track
Columns: PlaylistId, TrackId
Table: tracks
Columns: TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice
Table: sqlite_stat1
Columns: tbl, idx, stat


As before, we'll define a function specification for the function we'd like the API to generate arguments for. Notice that we are inserting the database schema into the function specification. This will be important for the model to know about.

In [74]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

### Executing SQL queries

Now let's implement the function that will actually excute queries against the database.

In [75]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

##### Steps to invoke a function call using Chat Completions API: 

**Step 1**: Prompt the model with content that may result in model selecting a tool to use. The description of the tools such as a function names and signature is defined in the 'Tools' list and passed to the model in API call. If selected, the function name and parameters are included in the response.<br>
  
**Step 2**: Check programmatically if model wanted to call a function. If true, proceed to step 3. <br>  
**Step 3**: Extract the function name and parameters from response, call the function with parameters. Append the result to messages. <br>    
**Step 4**: Invoke the chat completions API with the message list to get the response. 

In [76]:
# Step #1: Prompt with content that may result in function call. In this case the model can identify the information requested by the user is potentially available in the database schema passed to the model in Tools description. 
messages = [{
    "role":"user", 
    "content": "What is the name of the album with the most tracks?"
}]


In [30]:
if useOpenAI:
    response = client.chat.completions.create(
        # model='gpt-4o', 
        model=GPT_MODEL, 
        messages=messages, 
        tools= tools, 
        tool_choice="auto"
    )
    # Append the message to messages list
    response_message = response.choices[0].message 
    messages.append(response_message)
    print(response_message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_edLdFgTgb8iEp3VflYwrl83v', function=Function(arguments='{"query":"SELECT albums.Title FROM albums JOIN (SELECT tracks.AlbumId, COUNT(*) AS track_count FROM tracks GROUP BY tracks.AlbumId ORDER BY track_count DESC LIMIT 1) AS max_tracks ON max_tracks.AlbumId = albums.AlbumId"}', name='ask_database'), type='function')])


In [77]:
if not useOpenAI:
    response = client.chat.completions.create(
        # model='gpt-4o', 
        model=GPT_MODEL, 
        messages=messages, 
        tools= tools, 
        tool_choice="auto"
    )
    # Append the message to messages list
    response_message = response.choices[0].message 
    messages.append(response_message)
    print(response_message)

ChatCompletionMessage(content='To determine the name of the album with the most tracks, I would need more information about your music collection or a specific artist you are interested in. Please provide additional details so I can help you find the desired information.', role='assistant', function_call=None, tool_calls=None)


In [31]:
# Step 2: determine if the response from the model includes a tool call.  
if useOpenAI: 
    tool_calls = response_message.tool_calls
    if tool_calls:
        # If true the model will return the name of the tool / function to call and the argument(s)  
        tool_call_id = tool_calls[0].id
        tool_function_name = tool_calls[0].function.name
        tool_query_string = eval(tool_calls[0].function.arguments)['query']
        
        # Step 3: Call the function and retrieve results. Append the results to the messages list.      
        if tool_function_name == 'ask_database':
            results = ask_database(conn, tool_query_string)
            
            messages.append({
                "role":"tool", 
                "tool_call_id":tool_call_id, 
                "name": tool_function_name, 
                "content":results
            })
            
            # Step 4: Invoke the chat completions API with the function response appended to the messages list
            # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
            model_response_with_function_call = client.chat.completions.create(
                # model="gpt-4o",
                model=GPT_MODEL,
                messages=messages,
            )  # get a new response from the model where it can see the function response
            print(model_response_with_function_call.choices[0].message.content)
        else: 
            print(f"Error: function {tool_function_name} does not exist")
    else: 
        # Model did not identify a function to call, result can be returned to the user 
        print(response_message.content) 

The album with the most tracks is "Greatest Hits".


In [38]:
# Step 2: determine if the response from the model includes a tool call.  
if not useOpenAI: 
    tool_calls = response_message.tool_calls
    if tool_calls:
        # If true the model will return the name of the tool / function to call and the argument(s)  
        tool_call_id = tool_calls[0].id
        tool_function_name = tool_calls[0].function.name
        tool_query_string = eval(tool_calls[0].function.arguments)['query']
        
        # Step 3: Call the function and retrieve results. Append the results to the messages list.      
        if tool_function_name == 'ask_database':
            results = ask_database(conn, tool_query_string)
            
            messages.append({
                "role":"tool", 
                "tool_call_id":tool_call_id, 
                "name": tool_function_name, 
                "content":results
            })
            
            # Step 4: Invoke the chat completions API with the function response appended to the messages list
            # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
            model_response_with_function_call = client.chat.completions.create(
                # model="gpt-4o",
                model=GPT_MODEL,
                messages=messages,
            )  # get a new response from the model where it can see the function response
            print(model_response_with_function_call.choices[0].message.content)
        else: 
            print(f"Error: function {tool_function_name} does not exist")
    else: 
        # Model did not identify a function to call, result can be returned to the user 
        print(response_message.content) 

To determine the name of the album with the most tracks, I would need more information about your music collection or a specific artist you are interested in. Please provide additional details so I can help you find the desired information.


## Next Steps

See our other [notebook](How_to_call_functions_for_knowledge_retrieval.ipynb) that demonstrates how to use the Chat Completions API and functions for knowledge retrieval to interact conversationally with a knowledge base.